In [1]:
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import sounddevice as sd
import numpy as np
import json

In [2]:
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
print("volume.GetMute(): %s" % volume.GetMute())
print("volume.GetMasterVolumeLevel(): %s" % volume.GetMasterVolumeLevelScalar())
print("volume.GetVolumeRange(): (%s, %s, %s)" % volume.GetVolumeRange())
volume.SetMasterVolumeLevelScalar(0.22,None)
#print("volume.GetMasterVolumeLevel(): %s" % volume.GetMasterVolumeLevel())

volume.GetMute(): 0
volume.GetMasterVolumeLevel(): 0.24000000953674316
volume.GetVolumeRange(): (-65.25, 0.0, 0.03125)


0

In [3]:
from scipy.interpolate import interp1d

fs = 44100 // 2
duration = 1 #??
sig_len = fs * duration # sound length
array_len = 3675 
start_range, stop_range, step = volume.GetVolumeRange()
psoc_start = 48 # la psoc e mai mare rangeul dar poate merge asa
psoc_stop = 127
map_to_range = interp1d([psoc_start, psoc_stop],[start_range,stop_range]) # mapping from psoc to device range

def volume_denormalization(vol):
    return (psoc_stop - psoc_start) * vol + psoc_start

def volume_normalization(vol):
    #linear_vol = vol ** (vol / 10)
    #linear_start = psoc_start ** (vol / 10)
    #linear_stop = psoc_stop ** (vol / 10)
    #return (linear_vol - linear_start) / (linear_stop - linear_start)
    return (vol - psoc_start) / (psoc_stop - psoc_start)

In [4]:
# loading from file
data_array = []
with open('rec_file.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        obj = json.loads(line.replace('\n',''))
        data_array.append(obj)

#print(data_array)

In [5]:
# snd = np.load('outside_data\\' + data_array[0]['outside_sound'])
# sd.play(snd[:44000])
# sd.wait()
#print(sd.query_devices())
import sys
np.set_printoptions(threshold=sys.maxsize)
data = sd.rec(duration * fs, samplerate=fs, channels=1, dtype=np.float32, blocking=True)
#sd.wait()
sd.play(data)
print(len(data))
print(data)
np.set_printoptions(threshold=False)

22050
[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [-3.05175781e-05]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-3.05175781e-05]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-3.05175781e-05]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 3.05175781e-05]
 [ 0.00000000e+00]
 [-3.05175781e-05]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.0

In [6]:
# loading the model
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model_loaded = keras.models.load_model("SmartWave.h5")

_input = ''
while _input != 'stop':
    _input = input('predict?[y / n][\'stop\' to stop]')
    if _input == 'y':
        curr_db = volume.GetMasterVolumeLevel()
        print(f'current volume: {curr_db}')
        curr_db = np.array(curr_db).reshape(1,1)
        data = sd.rec(duration * fs, samplerate=fs, channels=1)
        sd.wait()
        #data = data.reshape(44100 , 1)
        #data = np.asarray([data])
        #print(curr_db.shape, data.shape)
        for i in range(0, sig_len // array_len):
            to_pred = data[i * array_len: (i + 1) * array_len]
            #to_pred.reshape(1,-1,1)
            to_pred = np.asarray(to_pred)
            print(to_pred.shape)
            ret = model_loaded.predict(to_pred,batch_size=1)
            ret = ret.flatten()
            ret = map_to_range(volume_denormalization(ret))
            print(f'volume will be set to: {ret}')
            volume.SetMasterVolumeLevel(ret, None)



current volume: -22.295886993408203
(3675, 1)


ValueError: in user code:

    f:\SmartWave_saucumsecheama\env_bun\lib\site-packages\tensorflow\python\keras\engine\training.py:1478 predict_function  *
        return step_function(self, iterator)
    f:\SmartWave_saucumsecheama\env_bun\lib\site-packages\tensorflow\python\keras\engine\training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    f:\SmartWave_saucumsecheama\env_bun\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    f:\SmartWave_saucumsecheama\env_bun\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    f:\SmartWave_saucumsecheama\env_bun\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    f:\SmartWave_saucumsecheama\env_bun\lib\site-